In [1]:
#-*- coding:utf-8 -*-

In [2]:
import graphlab
import re
import pandas as pd
import string
import jieba.posseg as pseg
import jieba
from pyltp import Segmentor, Postagger, Parser

In [3]:
segmentor = Segmentor()
postagger = Postagger()
parser = Parser()
segmentor.load('E:/Python/pyltp/ltp_data/cws.model')
postagger.load('E:/Python/pyltp/ltp_data/pos.model')
parser.load('E:/Python/pyltp/ltp_data/parser.model')

In [4]:
data = graphlab.SFrame.read_csv('SogouM.txt', delimiter='\t', header=True, column_type_hints=[str, str, str, str, str])

This non-commercial license of GraphLab Create is assigned to guoxiuhe@nefu.edu.cn and will expire on April 02, 2017. For commercial licensing options, visit https://dato.com/buy/.


2016-05-17 16:54:52,296 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: C:\Users\heguoxiu\AppData\Local\Temp\graphlab_server_1463475271.log.0


Finished parsing file E:\Python\2Graduation-Project\SogouM.txt

Parsing completed. Parsed 10000 lines in 0.247015 secs.

In [5]:
data.head()

﻿time,user_id,query,result_click,url
00:00:00,2982199073774412,[360安全卫士],8 3,download.it.com.cn/softweb/software/firewall/a ...
00:00:00,07594220010824798,[哄抢救灾物资],1 1,news.21cn.com/social/daqian/2008/05/29/4777194 ...
00:00:00,5228056822071097,[75810部队],14 5,www.greatoo.com/greatoo_cn/list.asp?link_id=27 ...
00:00:00,6140463203615646,[绳艺],62 36,www.jd-cd.com/jd_opus/xx/200607/706.html ...
00:00:00,8561366108033201,[汶川地震原因],3 2,www.big38.net/
00:00:00,23908140386148713,[莫衷一是的意思],1 2,www.chinabaike.com/article/81/82/110/2007/2007 ...
00:00:00,1797943298449139,[星梦缘全集在线观看] ...,8 5,www.6wei.net/dianshiju/????\xa1\xe9|????do=index ...
00:00:00,00717725924582846,[闪字吧],1 2,www.shanziba.com/
00:00:00,41416219018952116,[霍震霆与朱玲玲照片] ...,2 6,bbs.gouzai.cn/thread-698736.html ...
00:00:00,9975666857142764,[电脑创业],2 2,ks.cn.yahoo.com/question/1307120203719.html ...


In [6]:
data['result_and_click'] = data['result_click'].apply(lambda x : re.split(r'\s+',x))

Less than 4 successfully started. Using only 3 workers.

All operations will proceed as normal, but lambda operations will not be able to use all available cores.

To help us diagnose this issue, please send the log file to product-feedback@dato.com.

(The location of the log file is printed at the start of the GraphLab server).

In [7]:
def getfirst(l):
    return int(l[0])
def getsecond(l):
    return int(l[1])

data['result_rank'] = data['result_and_click'].apply(lambda l: getfirst(l))
data['click_rank'] = data['result_and_click'].apply(lambda l: getsecond(l))

In [8]:
data['query'] = data['query'].apply(lambda x: str(x)).apply(lambda x: x.strip().lstrip('[').rstrip(']'))

In [9]:
def remove_tail(url):
    if url[-1] == '/':
        return url[:-1]
    else:
        return url[:]

In [10]:
data['url'] = data['url'].apply(remove_tail)

In [11]:
data

﻿time,user_id,query,result_click,url,result_and_click,result_rank
00:00:00,2982199073774412,360安全卫士,8 3,download.it.com.cn/softweb/software/firewall/a ...,"[8, 3]",8
00:00:00,07594220010824798,哄抢救灾物资,1 1,news.21cn.com/social/daqian/2008/05/29/4777194 ...,"[1, 1]",1
00:00:00,5228056822071097,75810部队,14 5,www.greatoo.com/greatoo_cn/list.asp?link_id=27 ...,"[14, 5]",14
00:00:00,6140463203615646,绳艺,62 36,www.jd-cd.com/jd_opus/xx/200607/706.html ...,"[62, 36]",62
00:00:00,8561366108033201,汶川地震原因,3 2,www.big38.net,"[3, 2]",3
00:00:00,23908140386148713,莫衷一是的意思,1 2,www.chinabaike.com/article/81/82/110/2007/2007 ...,"[1, 2]",1
00:00:00,1797943298449139,星梦缘全集在线观��� ...,8 5,www.6wei.net/dianshiju/????\xa1\xe9|????do=index ...,"[8, 5]",8
00:00:00,00717725924582846,闪字吧,1 2,www.shanziba.com,"[1, 2]",1
00:00:00,41416219018952116,霍震霆与朱玲玲照��� ...,2 6,bbs.gouzai.cn/thread-698736.html ...,"[2, 6]",2
00:00:00,9975666857142764,电脑创业,2 2,ks.cn.yahoo.com/question/1307120203719.html ...,"[2, 2]",2


In [13]:
data_words = []
data_poses = []
data_parsers = []
for query in data['query']:
    temp_words = segmentor.segment(query)
    temp_poses = postagger.postag(temp_words)
    temp_parsers = parser.parse(temp_words, temp_poses)
    words = ' '.join(temp_words)
    poses = ' '.join(temp_poses)
    parsers = ' '.join("%s" % (arc.relation) for arc in temp_parsers)
    data_words.append(words)
    data_poses.append(poses)
    data_parsers.append(parsers)

In [14]:
data['words'] = data_words
data['poses'] = data_poses
data['parsers'] = data_parsers

In [15]:
data.head()

﻿time,user_id,query,result_click,url,result_and_click,result_rank
00:00:00,2982199073774412,360安全卫士,8 3,download.it.com.cn/softweb/software/firewall/a ...,"[8, 3]",8
00:00:00,07594220010824798,哄抢救灾物资,1 1,news.21cn.com/social/daqian/2008/05/29/4777194 ...,"[1, 1]",1
00:00:00,5228056822071097,75810部队,14 5,www.greatoo.com/greatoo_cn/list.asp?link_id=27 ...,"[14, 5]",14
00:00:00,6140463203615646,绳艺,62 36,www.jd-cd.com/jd_opus/xx/200607/706.html ...,"[62, 36]",62
00:00:00,8561366108033201,汶川地震原因,3 2,www.big38.net,"[3, 2]",3
00:00:00,23908140386148713,莫衷一是的意思,1 2,www.chinabaike.com/article/81/82/110/2007/2007 ...,"[1, 2]",1
00:00:00,1797943298449139,星梦缘全集在线观��� ...,8 5,www.6wei.net/dianshiju/????\xa1\xe9|????do=index ...,"[8, 5]",8
00:00:00,00717725924582846,闪字吧,1 2,www.shanziba.com,"[1, 2]",1
00:00:00,41416219018952116,霍震霆与朱玲玲照��� ...,2 6,bbs.gouzai.cn/thread-698736.html ...,"[2, 6]",2
00:00:00,9975666857142764,电脑创业,2 2,ks.cn.yahoo.com/question/1307120203719.html ...,"[2, 2]",2


In [16]:
data['len_words'] = data['query'].apply(lambda x: len(x.decode('utf-8')))

In [17]:
data['len_seg'] = data['words'].apply(lambda x: len(re.split(r'\s+',x)))

In [18]:
data['len_parser'] = data['parsers'].apply(lambda x: len(re.split(r'\s+',x)))

In [19]:
data.head()

﻿time,user_id,query,result_click,url,result_and_click,result_rank
00:00:00,2982199073774412,360安全卫士,8 3,download.it.com.cn/softweb/software/firewall/a ...,"[8, 3]",8
00:00:00,07594220010824798,哄抢救灾物资,1 1,news.21cn.com/social/daqian/2008/05/29/4777194 ...,"[1, 1]",1
00:00:00,5228056822071097,75810部队,14 5,www.greatoo.com/greatoo_cn/list.asp?link_id=27 ...,"[14, 5]",14
00:00:00,6140463203615646,绳艺,62 36,www.jd-cd.com/jd_opus/xx/200607/706.html ...,"[62, 36]",62
00:00:00,8561366108033201,汶川地震原因,3 2,www.big38.net,"[3, 2]",3
00:00:00,23908140386148713,莫衷一是的意思,1 2,www.chinabaike.com/article/81/82/110/2007/2007 ...,"[1, 2]",1
00:00:00,1797943298449139,星梦缘全集在线观��� ...,8 5,www.6wei.net/dianshiju/????\xa1\xe9|????do=index ...,"[8, 5]",8
00:00:00,00717725924582846,闪字吧,1 2,www.shanziba.com,"[1, 2]",1
00:00:00,41416219018952116,霍震霆与朱玲玲照��� ...,2 6,bbs.gouzai.cn/thread-698736.html ...,"[2, 6]",2
00:00:00,9975666857142764,电脑创业,2 2,ks.cn.yahoo.com/question/1307120203719.html ...,"[2, 2]",2


In [20]:
data.save('Sogou1W_ltp.csv')